# Compound Matrix Method Evans Function

### Note: Define all needed symbols outside the class

In [1]:
from sympy import *
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from scipy import linalg as la
from time import time

In [2]:
from IPython.display import display, Markdown, Latex

In [3]:
plt.rcParams['figure.figsize'] = 15, 5

In [4]:
G, nu, lam, s, a0, a1, a2, a3, a4, a5 = symbols('G, nu, lam, s, a0, a1, a2, a3, a4, a5')
alp, bet = symbols('alp, bet')
fsig = alp+bet*s
mu = 1

In [5]:
vp = G/(G+2)
ep = 2/(G+2)**2

e_coeffs = [ep,a1,a2,a3,a4,a5]
e = sympify(0)
for j in range(0,len(e_coeffs)):
    e = e+e_coeffs[j]*s**j

In [6]:
v = s + vp
vx = (v*(v-1)+G*e)/mu
ex = v*(-.5*(v-1)**2+e)/nu
ux = vx
uxx = (vx*(2*v-1)+G*ex)/mu

g = (G*e-(nu+1)*ux)/nu
h = -ex/v
f = 2*v - 1

In [7]:
def convertMatrix(matrix, vector):
    temp_matrix = []
    for i in range(matrix.shape[0]):
        temp_row = []
        for j in range(matrix.shape[1]):
            temp_val = matrix[10*i+j]
            temp_row.append(temp_val.simplify())
        temp_matrix.append(temp_row)
    new_matrix = Matrix(temp_matrix)
        
    temp_vector = []
    for i in range(vector.shape[0]):
        temp_row = []
        for j in range(vector.shape[1]):
            temp_val = vector[i+j]
            temp_row.append(temp_val.simplify())
        temp_vector.append(temp_row)
    new_vector = Matrix(temp_vector)
    
    return new_matrix, new_vector

In [8]:
class SystemSolve():
    def __init__(self,dim,terms,A,deriv,e_val,print_on=True, comp=False):
        self.dim = dim
        self.terms = terms
        self.A = A
        self.deriv = deriv
        self._createB()
        self.p = print_on
        self.comp = comp
        self._setEigs(e_val)
        return
    
    def _createB(self):
        self.B = []
        for i in range(self.terms):
            b = []
            for j in range(1,self.dim+1):
                b.append(sympify('B{}_{}'.format(i,j)))
            self.B.append(b)
        return
    
    def _setEigs(self, e_val):
        self.e_val = e_val
        self.B_tups = []
        return
    
    def createSys(self):
        W = []
        W_deriv = []

        for j in range(self.dim):
            w = sympify(0)
            for i in range(self.terms):
                w += self.B[i][j]*s**i
            w_deriv = diff(w, s)

            W.append(w)
            W_deriv.append(w_deriv)

        # Store W and W' as numpy arrays
        self.W = np.array(W)
        self.W_deriv = np.array(W_deriv)

        # Replace matrix A by A-lam*I
        Anew = self.A - np.eye(self.dim)*self.e_val
        
        # Define system of equations from each row of A
        eqs = []
        for i in range(self.dim):
            eq = np.dot(Anew[i],self.W)-self.W_deriv[i]*self.deriv
            #print(eq, '\n\n\n')
            eqs.append(eq)
   
        # Isolate coefficients for each power of sigma
        eq_der = [eq.copy() for eq in eqs]
        heqs = []

        for i in range(self.terms):
            if i==0: prod = 1
            else: 
                prod *= i

            if self.p: print('\n\npower = {}'.format(i)) 
            res = [(eqd.subs(s,0)/prod).simplify() for eqd in eq_der]
            #res = [eqd.subs(s,0)/prod for eqd in eq_der]

            if self.p:
                for r in res:
                    print('\n')
                    print(r)
                    #print('\n')
                    #print(r.simplify())
                    
            res = [r.expand() for r in res]

            heqs.append(res)    
            eq_der = [diff(eqd,s) for eqd in eq_der]
            
        self.heqs = heqs
        return 
    
    def wCoefs(self):
        heqs = self.heqs
        # Solve for the coefficients of W
        up = 0
        stop = False
        
        # If there are issues uncomment these two lines
        #up = 1
        #stop = True
        start = time()
        for j in range(len(heqs)):
            if j == len(heqs)-1 and stop:
                break
            if self.p:
                p1 = '-'*21+'\nsystem when sigma = '
                p2 = str(j)+'\n'+'-'*21+'\n'
                print(p1+p2)
                
            if j == 0:
                try: 
                    matrix, vector = linear_eq_to_matrix(heqs[j],self.B[j+up])
                    soln = matrix.LUsolve(vector)
                    #print('made it here')
                except: 
                    print('failed')
                    up = 0
                    stop = False
            
            #print('solving')
            #print(heqs[j],'\n')
            print('solving for',self.B[j+up],'\n')
            matrix, vector = linear_eq_to_matrix(heqs[j],self.B[j+up])
            #print(matrix[:10])
            
            matrix, vector = convertMatrix(matrix, vector)
            #print(matrix[:10])
            
            #matrix_inv = matrix.inv()
            
            print('Matrix')
            print(matrix,'\n')
            print('Vector')
            print(vector,'\n')
            
            
            if self.comp:
                vector = np.array([complex(vector[i].subs(self.B_tups)) for i in range(self.dim)])
                matrix = np.array([[complex(matrix[i,j].subs(self.B_tups)) for j in range(self.dim)] for i in range(self.dim)])
            
            #matrix = np.array(matrix.tolist()).astype(complex)
            #vector = np.array(vector.tolist()).astype(complex)

            if self.comp:
                soln = np.linalg.solve(matrix,vector)
            else:
                soln = matrix.LUsolve(vector)

            for i in range(self.dim):
                if self.comp: 
                    val = soln[i]
                    val = complex(val)
                else:
                    val = soln[i].subs(self.B_tups)
                
    
                #if j != 0 or stop: 
                #    self.B_tups.append((self.B[j+up][i],val))
                """if self.p: 
                    print('$$')
                    print(self.B[j+up][i])
                    print('=')
                    if j==0: print(val)
                    else: print(len(val))
                    print('$$')
                    print('\n\n')
                    
                    #val = '$$'+str(self.B[j+up][i]) +' = '+str(val)+'$$'
                    #print('about to print')
                    #print(val,'\n\n')"""

## Evans Function

In [9]:
# for this one the eigenvalue matches
def lift_A(a):
    to_return = [[a[0,0] + a[1,1] + a[2,2], a[2,3], a[2,4], -a[1,3], -a[1,4], 0, a[0,3], a[0,4], 0, 0],\
                 [a[3,2], a[0,0] + a[1,1] + a[3,3], a[3,4], a[1,2], 0, -a[1,4], -a[0,2], 0, a[0,4], 0],\
                 [a[4,2], a[4,3], a[0,0] + a[1,1] + a[4,4], 0, a[1,2], a[1,3], 0, -a[0,2], -a[0,3], 0],\
                 [-a[3,1], a[2,1], 0, a[0,0] + a[2,2] + a[3,3], a[3,4], -a[2,4], a[0,1], 0, 0, a[0,4]],\
                 [-a[4,1], 0, a[2,1], a[4,3], a[0,0] + a[2,2] + a[4,4], a[2,3], 0, a[0,1], 0, -a[0,3]],\
                 [0, -a[4,1], a[3,1], -a[4,2], a[3,2], a[0,0] + a[3,3] + a[4,4], 0, 0, a[0,1], a[0,2]],\
                 [a[3,0], -a[2,0], 0, a[1,0], 0, 0, a[1,1] + a[2,2] + a[3,3], a[3,4], -a[2,4], a[1,4]],\
                 [a[4,0], 0, -a[2,0], 0, a[1,0], 0, a[4,3], a[1,1] + a[2,2] + a[4,4], a[2,3], -a[1,3]],\
                 [0, a[4,0], -a[3,0], 0, 0, a[1,0], -a[4,2], a[3,2], a[1,1] + a[3,3] + a[4,4], a[1,2]],\
                 [0, 0, 0, a[4,0], -a[3,0], a[2,0], a[4,1], -a[3,1], a[2,1], a[2,2] + a[3,3] + a[4,4]]]
    return to_return

In [10]:
def create_lifted_evans():
    # define parameters

    A = np.array([[0,1,0,0,0],\
                  [lam*v/nu,v/nu,v*ux/nu-uxx,lam*g,g-h],\
                  [0,0,0,lam,1],\
                  [0,0,0,0,1],\
                  [0,G,lam*v+G*ux,lam*v,f-lam]])
    
    A_lift = lift_A(A)
    return A_lift

In [11]:
A = create_lifted_evans()

In [12]:
sys = SystemSolve(10, 5, A, vx, fsig)
sys.createSys()



power = 0


(B0_1*(G + 2)*(G - 1.0*alp*nu*(G + 2)) - 2*G*(B0_4*lam + B0_5) + nu*(G + 2)**2*(B0_2*lam + B0_3))/(nu*(G + 2)**2)


(B0_2*(G + 2)*(G - 1.0*alp*nu*(G + 2)) + B0_3*nu*(G + 2)**2 - 2*B0_6*G)/(nu*(G + 2)**2)


(-B0_3*(G + 2)*(-2*G*nu - G + nu*(G + 2)*(1.0*alp + lam + 1)) + 2*B0_6*G*lam + G*lam*nu*(B0_1 + B0_2)*(G + 2))/(nu*(G + 2)**2)


-1.0*B0_4*alp + B0_5 - B0_6 + B0_7


(B0_4*G*lam - B0_5*(-2*G + (G + 2)*(1.0*alp + lam + 1)) + (G + 2)*(-B0_1*G + B0_6*lam + B0_8))/(G + 2)


(-B0_4*G*lam - B0_6*(-2*G + (G + 2)*(1.0*alp + lam + 1)) + (G + 2)*(-B0_2*G + B0_9))/(G + 2)


(2*B0_10*G + nu*(B0_8 - B0_9)*(G + 2)**2 + (G + 2)*(B0_4*G*lam + B0_7*(G - 1.0*alp*nu*(G + 2))))/(nu*(G + 2)**2)


(-2*B0_10*G*lam + B0_5*G*lam*(G + 2) + B0_7*G*lam*nu*(G + 2) - B0_8*(G + 2)*(-2*G*nu - G + nu*(G + 2)*(1.0*alp + lam + 1)) + B0_9*lam*nu*(G + 2)**2)/(nu*(G + 2)**2)


(B0_6*G*lam - B0_7*G*lam*nu - B0_9*(-2*G*nu - G + nu*(G + 2)*(1.0*alp + lam + 1)))/(nu*(G + 2))


(-B0_10*(-2*G + (G + 2)*(1.0*alp +

In [13]:
soln = sys.wCoefs()

---------------------
system when sigma = 0
---------------------

solving for [B0_1, B0_2, B0_3, B0_4, B0_5, B0_6, B0_7, B0_8, B0_9, B0_10] 

Matrix
Matrix([[(-1.0*G*alp*nu + 1.0*G - 2.0*alp*nu)/(nu*(1.0*G + 2.0)), lam, 1, -2*G*lam/(nu*(G**2 + 4*G + 4)), -2*G/(nu*(G**2 + 4*G + 4)), 0, 0, 0, 0, 0], [0, (-1.0*G*alp*nu + 1.0*G - 2.0*alp*nu)/(nu*(1.0*G + 2.0)), 1, 0, 0, -2*G/(nu*(G**2 + 4*G + 4)), 0, 0, 0, 0], [G*lam/(G + 2), G*lam/(G + 2), (-1.0*G*alp*nu - 1.0*G*lam*nu + 1.0*G*nu + 1.0*G - 2.0*alp*nu - 2.0*lam*nu - 2.0*nu)/(nu*(1.0*G + 2.0)), 0, 0, 2*G*lam/(nu*(G**2 + 4*G + 4)), 0, 0, 0, 0], [0, 0, 0, -1.0*alp, 1, -1, 1, 0, 0, 0], [-G, 0, 0, G*lam/(G + 2), (-1.0*G*alp - G*lam + G - 2.0*alp - 2*lam - 2)/(G + 2), lam, 0, 1, 0, 0], [0, -G, 0, -G*lam/(G + 2), 0, (-1.0*G*alp - G*lam + G - 2.0*alp - 2*lam - 2)/(G + 2), 0, 0, 1, 0], [0, 0, 0, G*lam/(nu*(G + 2)), 0, 0, (-1.0*G*alp*nu + 1.0*G - 2.0*alp*nu)/(nu*(1.0*G + 2.0)), 1, -1, 2*G/(nu*(G**2 + 4*G + 4))], [0, 0, 0, 0, G*lam/(nu*(G + 2)), 0, 